# Retrieval Question/Answering

This example showcases question answering over an index built from a folder of PDFs, returning the sources and splitting with NLTK. 

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI


Read all pdf files from the folder:
"G:\.shortcut-targets-by-id\1vE28d8xZuJXkpcinFbuku9FJgeaDd48K\ICOLD - CFRD New Bulletin 2023"

In [4]:
folder = r"G:\.shortcut-targets-by-id\1vE28d8xZuJXkpcinFbuku9FJgeaDd48K\ICOLD - CFRD New Bulletin 2023"

In [5]:
from langchain.document_loaders import DirectoryLoader

In [6]:
# Load all PDFs in a folder recursively
loader = DirectoryLoader(folder, "**/*.pdf")

In [7]:
docs = loader.load()

detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.
detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.
detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.
detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.
detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.
detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.
detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.
detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the

In [8]:
len(    docs)

66

In [9]:
from langchain.text_splitter import NLTKTextSplitter
text_splitter = NLTKTextSplitter(chunk_size=2000)

In [10]:
texts = text_splitter.split_documents(docs)

Created a chunk of size 5172, which is longer than the specified 2000
Created a chunk of size 2857, which is longer than the specified 2000
Created a chunk of size 3047, which is longer than the specified 2000
Created a chunk of size 3195, which is longer than the specified 2000
Created a chunk of size 2456, which is longer than the specified 2000
Created a chunk of size 3227, which is longer than the specified 2000
Created a chunk of size 3158, which is longer than the specified 2000
Created a chunk of size 2527, which is longer than the specified 2000
Created a chunk of size 2583, which is longer than the specified 2000
Created a chunk of size 3447, which is longer than the specified 2000
Created a chunk of size 2735, which is longer than the specified 2000
Created a chunk of size 3250, which is longer than the specified 2000
Created a chunk of size 2473, which is longer than the specified 2000
Created a chunk of size 2970, which is longer than the specified 2000
Created a chunk of s

In [11]:
print(texts[10].page_content)

Types de fissures dans la dalle du masque ......................................

6.3.2.

Maîtrise des fissures de la dalle ........................................................

6.4.

Propriétés du béton ......................................................................................

6.4.1.

Propriétés du mélange .......................................................................

6.4.2.

Granulats du béton ............................................................................

6.5.

Armature .......................................................................................................

6.5.1.

Pourcentages d’armature..................................................................

6.5.2.

Espacement des barres et recouvrement de béton .......................

6.5.3.

Acier anti-éclatement .......................................................................

6.5.4.

Armature de fibre ....................................................................

In [12]:
print(f"Number of chunks: {len(texts)}")

Number of chunks: 2626


In [13]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()
# docsearch = Chroma.from_documents(texts, embeddings)

In [14]:
# Set up Pinecone
import pinecone
pinecone.init(api_key="48640420-7e79-46d4-b71d-d07286818fef", environment="us-central1-gcp")

c:\Users\juantello\Anaconda3\envs\langchain\Lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [15]:
embeddings

OpenAIEmbeddings(client=<class 'openai.api_resources.embedding.Embedding'>, model='text-embedding-ada-002', document_model_name='text-embedding-ada-002', query_model_name='text-embedding-ada-002', embedding_ctx_length=8191, openai_api_key=None, openai_organization=None, chunk_size=1000, max_retries=6)

In [16]:
# check if 'icold' exists in the indexes
if 'icold' in pinecone.list_indexes():
    # Delete the index if it already exists
    pinecone.delete_index('icold')
# create a new index
pinecone.create_index('icold', dimension=1536, metric='cosine')

In [17]:
# Upsert the documents into the index
from tqdm.autonotebook import tqdm
from langchain.vectorstores import Pinecone

In [18]:
[s for s in dir(Pinecone) if not s.startswith("_")]

['aadd_documents',
 'aadd_texts',
 'add_documents',
 'add_texts',
 'afrom_documents',
 'afrom_texts',
 'amax_marginal_relevance_search',
 'amax_marginal_relevance_search_by_vector',
 'as_retriever',
 'asimilarity_search',
 'asimilarity_search_by_vector',
 'from_documents',
 'from_existing_index',
 'from_texts',
 'max_marginal_relevance_search',
 'max_marginal_relevance_search_by_vector',
 'similarity_search',
 'similarity_search_by_vector',
 'similarity_search_with_score']

In [19]:
docsearch = Pinecone.from_documents(texts, embeddings, index_name='icold')

In [20]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0.5, max_tokens=2000),
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
    return_source_documents=True,
    )

In [21]:
def pretty_print(response):
    import textwrap
    # Split the response by lines of max 80 characters
    return '\n'.join(textwrap.wrap(response, 80))


In [22]:
query = "tell me about campos novos dam"
result = qa({"query": query, "n": 10})

InvalidRequestError: This model's maximum context length is 4097 tokens. However, you requested 4128 tokens (2128 in the messages, 2000 in the completion). Please reduce the length of the messages or completion.

In [ ]:
print(pretty_print(result['result']))

The methodology described in the given context has been used to reproduce
various aspects of the behavior observed in different dams around the world,
including Campos Novos Dam in Brazil. However, the given context does not
provide detailed information about Campos Novos Dam. If you need more
information about it, please let me know, and I will try to help you find it.


In [ ]:
result['source_documents'][3].metadata['source']

'G:\\.shortcut-targets-by-id\\1vE28d8xZuJXkpcinFbuku9FJgeaDd48K\\ICOLD - CFRD New Bulletin 2023\\B141_unlocked.pdf'

In [ ]:
query = "What do you know about the Campos Novos Dam?"
result = qa({"query": query, "n": 10})
print(result['result'])

According to the context, the Campos Novos Dam is a dam that has been studied using the methodology developed by INGETEC. The methodology has been consistently reproduced in various aspects of behavior observed in different dams around the world, including the Campos Novos Dam in Brazil. However, there is no further information provided about the specific details of the Campos Novos Dam.


In [ ]:
pretty_print(result['result'])

'According to the context, the Campos Novos Dam is a dam that has been studied\nusing the methodology developed by INGETEC. The methodology has been\nconsistently reproduced in various aspects of behavior observed in different\ndams around the world, including the Campos Novos Dam in Brazil. However, there\nis no further information provided about the specific details of the Campos\nNovos Dam.'

In [ ]:
result['source_documents'][3].metadata['source']

'G:\\.shortcut-targets-by-id\\1vE28d8xZuJXkpcinFbuku9FJgeaDd48K\\ICOLD - CFRD New Bulletin 2023\\Dam Response\\sdar.64119.381.pdf'